<a href="https://colab.research.google.com/github/ujeongmin/ESAA/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82_%EC%A0%95%EB%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 라이브러리, 데이터 불러오기 (공통 부분)

In [55]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
from google.colab import drive
drive.mount('/content/gdrive')

test = pd.read_csv('/content/gdrive/MyDrive/ESAA/data/wine/test.csv')
train = pd.read_csv('/content/gdrive/MyDrive/ESAA/data/wine/train.csv')
sub = pd.read_csv('/content/gdrive/MyDrive/ESAA/data/wine/sample_submission.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [57]:
# index 칼럼 드랍
train = train.drop('index', axis = 1)
test = test.drop('index', axis = 1)

# 라벨 인코딩 > white와 red를 각각 0, 1로 변환
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])



---



## XGBoost

### 데이터 정규화 시 오히려 전반적으로 점수가 하락해서 정규화 없이 진행하기로 함
* XGBoost는 트리 기반 모델이기 때문에 피처 스케일링과 정규화가 필수적이지 않음. 데이터 크기보다는 대소 관계의 영향을 받기 때문

In [58]:
from xgboost import XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np

In [59]:
train1 = train.copy()
test1 = test.copy()

train_x = train1.drop('quality', axis = 1)       # 학습 피처 데이터 세트
train_y = train1['quality']                      # 학습 레이블 데이터 세트
test_x = test1                                   # 테스트 피처 데이터 세트

In [60]:
le = LabelEncoder()
train_y = le.fit_transform(train_y)

In [61]:
# 하이퍼 파라미터

from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(n_estimators=100)

params = {'max_depth': [13,14,15] , 'min_child_weight': [1,2] ,'colsample_bytree': [0.5,0.75]}

gridcv = GridSearchCV(xgb, param_grid=params, scoring='accuracy', cv=5, verbose=1)
gridcv.fit(train_x, train_y)

print('GridSearchCV 최고 평균 정확도 수치: {0:.4f}'.format(gridcv.best_score_))
print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
GridSearchCV 최고 평균 정확도 수치: 0.6624
GridSearchCV 최적 파라미터: {'colsample_bytree': 0.75, 'max_depth': 14, 'min_child_weight': 1}


In [ ]:
model = XGBClassifier(n_estimator=1000, max_depth=14, min_child_weight=1, colsample_bytree=0.75)

model.fit(train_x, train_y, eval_metric='logloss')
y_pred = model.predict(test_x)

### 점수: 0.693 ###

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:47:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)



### 정확도 개선 위해 기존 파라미터에 learning_rate, reg_alpha 추가하고 GridSearch 수행


In [62]:
# 하이퍼 파라미터

from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(n_estimators=100)

params = {'max_depth': [14] , 'min_child_weight': [1] ,'colsample_bytree': [0.75], 'learning_rate':[0.01,0.04,0.06],'reg_alpha':[0.01,0.03,0.05]}

gridcv = GridSearchCV(xgb, param_grid=params, scoring='accuracy', cv=5, verbose=1)
gridcv.fit(train_x, train_y)

print('GridSearchCV 최고 평균 정확도 수치: {0:.4f}'.format(gridcv.best_score_))
print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
GridSearchCV 최고 평균 정확도 수치: 0.6693
GridSearchCV 최적 파라미터: {'colsample_bytree': 0.75, 'learning_rate': 0.04, 'max_depth': 14, 'min_child_weight': 1, 'reg_alpha': 0.03}


In [ ]:
model = XGBClassifier(n_estimator=1000, max_depth=14, min_child_weight=1, colsample_bytree=0.75, learning_rate=0.04, reg_alpha=0.03)

model.fit(train_x, train_y, eval_metric='logloss')
y_pred = model.predict(test_x)

### 점수: 0.702 ###

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:18:21] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


### 마지막으로 파라미터 값들 직접 조정하며 점수 확인해본 결과, max_depth를 13으로 설정하는 것이 기존보다 0.001 더 높은 점수를 냄

In [36]:
model = XGBClassifier(n_estimator=1000, max_depth=13, min_child_weight=1, colsample_bytree=0.75, learning_rate=0.04, reg_alpha=0.03)

model.fit(train_x, train_y, eval_metric='logloss')
y_pred = model.predict(test_x)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:58:49] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


In [35]:
# submission 파일 저장

sub['quality'] = y_pred + 3
sub.to_csv('/content/gdrive/MyDrive/ESAA/data/wine/submission.csv', index=False)

### 최종 점수: 0.703 ###